In [ ]:
#!/usr/bin/python
# coding: utf-8

from datetime import date, datetime, timedelta
from predict_functions import *
from sklearn.externals import joblib
import pandas as pd

# TODO 0: installer et configurer treetagger sur le serveur

# TODO 1: integrer le code qui permet de recuperer les tweets pour un candidat et un jour donnés
# (par tranche d'un jour? trois jours pour lisser les résultats?) sous forme de dataframe avec une colonne 'text'

# TODO 2: transferer dans un fichier python, upload sur le serveur et programmer un cron (une fois par jour?)

In [ ]:
today = (datetime.utcnow() + timedelta(hours=2)).date() # heure d'été, on est en UTC+2 au lieu de +1
fname = 'twitter_sentiments_{}.json'.format(today)

df_pred = pd.DataFrame()
candidates = ['macron', 'fillon']

# 1. Chargement du vocabulaire
voca = pd.read_json('trained_dict.json').to_dict()[0]

for candidate in candidates:
    # 2. Chargement des tweets a predire
    print('Chargement des tweets du candidat {}'.format(candidate))
    # df_tweets = .... (dataframe contenant les tweets a predire dans une colonne 'text', creer un df par candidat)

    # 3. Creation des features et de la matrice TF-IDF pour la base test
    X_test, _, _ = build_Xy(df_tweets, drop_dups=False, vocab=voca, min_df=3, n_grams=(1,1))

    # 4. Chargement du modele entraine
    clf = joblib.load('trained_logistic_regression.pkl')

    # 5. Prediction
    y_pred = clf.predict(X_test)
    results = [np.sum(y_pred==label) / len(y_pred) for label in [1, 0, -1]]

    # ajout de la ligne du candidat dans le dataframe
    rec = pd.DataFrame([results], columns=['pos', 'neu', 'neg'], index=[candidate])
    df_pred = df_pred.append(rec, verify_integrity=False)

# 6. Sauvegarder les predictions (dans un dataframe puis un json par exemple?)
df_pred.to_json(fname)